# Quadrotor Example
This notebook will demonstrate how to set up and solve a trajectory optimization problem for a quadrotor. In particular, it will highlight how TrajectoryOptimization.jl accounts for the group structure of 3D rotations.

### Loading the Required Packages
To define the quadrotor model, we import `RobotDynamics` and `Rotations`, and use `TrajectoryOptimization` to define the problem. We load in `StaticArrays` and `LinearAlgebra` to help with the setup.

In [1]:
using RobotDynamics, Rotations
using TrajectoryOptimization
using StaticArrays, LinearAlgebra

## Creating the model
We could use the quadrotor model defined in `RobotZoo.jl`, but instead we'll go through the details of using the `RigidBody` interface in `RobotDyanmics`.

We start by defining our new `Quadrotor` type, which inherits from `RigidBody{R}`:

In [2]:
struct Quadrotor{R} <: RigidBody{R}
    n::Int
    m::Int
    mass::Float64
    J::Diagonal{Float64,SVector{3,Float64}}
    Jinv::Diagonal{Float64,SVector{3,Float64}}
    gravity::SVector{3,Float64}
    motor_dist::Float64
    kf::Float64
    km::Float64
    bodyframe::Bool  # velocity in body frame?
    info::Dict{Symbol,Any}
end
control_dim(::Quadrotor) = 4

function Quadrotor{R}(;
        mass=0.5,
        J=Diagonal(@SVector [0.0023, 0.0023, 0.004]),
        gravity=SVector(0,0,-9.81),
        motor_dist=0.1750,
        kf=1.0,
        km=0.0245,
        bodyframe=false,
        info=Dict{Symbol,Any}()) where R
    Quadrotor{R}(13,4,mass,J,inv(J),gravity,motor_dist,kf,km,bodyframe,info)
end

(::Type{Quadrotor})(;kwargs...) = Quadrotor{UnitQuaternion{Float64}}(;kwargs...)

where `R` is the rotation parameterization being used, typically one of `UnitQuaternion{T}`, `MRP{T}`, or `RodriguesParam{T}`. 

We now need to define the number of control inputs:

In [3]:
RobotDynamics.control_dim(::Quadrotor) = 4

Now we are ready to define the dynamics of our quadrotor, which we do by simply defining the forces and moments acting on our quadrotor for a given state and control, as well as some "getter" methods for our inertial properties.

It's important to note that the force is in the world frame, and torque is in the body frame.

In [4]:
function RobotDynamics.forces(model::Quadrotor, x, u)
    q = orientation(model, x)
    kf = model.kf
    g = model.gravity
    m = model.mass

    # Extract motor speeds
    w1 = u[1]
    w2 = u[2]
    w3 = u[3]
    w4 = u[4]

    # Calculate motor forces
    F1 = max(0,kf*w1);
    F2 = max(0,kf*w2);
    F3 = max(0,kf*w3);
    F4 = max(0,kf*w4);
    F = @SVector [0., 0., F1+F2+F3+F4] #total rotor force in body frame

    m*g + q*F # forces in world frame
end

function RobotDynamics.moments(model::Quadrotor, x, u)

    kf, km = model.kf, model.km
    L = model.motor_dist

    # Extract motor speeds
    w1 = u[1]
    w2 = u[2]
    w3 = u[3]
    w4 = u[4]
    
    # Calculate motor forces
    F1 = max(0,kf*w1);
    F2 = max(0,kf*w2);
    F3 = max(0,kf*w3);
    F4 = max(0,kf*w4);

    # Calculate motor torques
    M1 = km*w1;
    M2 = km*w2;
    M3 = km*w3;
    M4 = km*w4;
    tau = @SVector [L*(F2-F4), L*(F3-F1), (M1-M2+M3-M4)] #total rotor torque in body frame
end

RobotDynamics.inertia(model::Quadrotor) = model.J
RobotDynamics.inertia_inv(model::Quadrotor) = model.Jinv
RobotDynamics.mass(model::Quadrotor) = model.mass

And with that our model is defined!

## Setting up our problem
For our trajectory optimization problem, we're going to have the quadrotor do a "zig-zag" pattern. We can do this via objective/cost function manipulation. We start by creating our quadrotor model and defining our integration scheme:

In [5]:
# Set up model and discretization
model = Quadrotor();
n,m = size(model)
N = 101                # number of knot points
tf = 5.0               # total time (sec)
dt = tf/(N-1)          # time step (sec)

0.05

We now need to set up the initial and final conditions for our quadrotor, which we want to move 20 meters in the x-direction. We can build the state piece-by-piece using the `RobotDynamics.build_state` function.

In [6]:
x0_pos = SA[0, -10, 1.]
xf_pos = SA[0, +10, 1.]
x0 = RobotDynamics.build_state(model, x0_pos, UnitQuaternion(I), zeros(3), zeros(3))
xf = RobotDynamics.build_state(model, xf_pos, UnitQuaternion(I), zeros(3), zeros(3));

### Creating the cost function
We now create a cost function that encourages a "zig-zag" pattern for the quadrotor. We set up a few waypoints at specific times, and impose a high cost for being far from those locations.

In [7]:
# Set up waypoints
wpts = [SA[+10, 0, 1.],
        SA[-10, 0, 1.],
        xf_pos]
times = [33, 66, 101]   # in knot points

# Set up nominal costs
Q = Diagonal(RobotDynamics.fill_state(model, 1e-5, 1e-5, 1e-3, 1e-3))
R = Diagonal(@SVector fill(1e-4, 4))
q_nom = UnitQuaternion(I)
v_nom = zeros(3)
ω_nom = zeros(3)
x_nom = RobotDynamics.build_state(model, zeros(3), q_nom, v_nom, ω_nom)
cost_nom = LQRCost(Q, R, x_nom)

# Set up waypoint costs
Qw_diag = RobotDynamics.fill_state(model, 1e3,1,1,1)
Qf_diag = RobotDynamics.fill_state(model, 10., 100, 10, 10)
costs = map(1:length(wpts)) do i
    r = wpts[i]
    xg = RobotDynamics.build_state(model, r, q_nom, v_nom, ω_nom)
    if times[i] == N
        Q = Diagonal(Qf_diag)
    else
        Q = Diagonal(1e-3*Qw_diag)
    end

    LQRCost(Q, R, xg)
end

# Build Objective
costs_all = map(1:N) do k
    i = findfirst(x->(x ≥ k), times)
    if k ∈ times
        costs[i]
    else
        cost_nom
    end
end
obj = Objective(costs_all);

### Initialization
We initialize the solver with a simple hover trajectory that keeps the quadrotor hovering at the initial position.

In [8]:
u0 = @SVector fill(0.5*model.mass/m, m)
U_hover = [copy(u0) for k = 1:N-1]; # initial hovering control trajectory

### Constraints
For this problem, we only impose bounds on the controls.

In [9]:
conSet = ConstraintList(n,m,N)
bnd = BoundConstraint(n,m, u_min=0.0, u_max=12.0)
add_constraint!(conSet, bnd, 1:N-1)

### Building the Problem
We now build the trajectory optimization problem, providing a dynamically-feasible initialization.

In [10]:
prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet)
initial_controls!(prob, U_hover)
rollout!(prob);

## Solving the Problem using ALTRO
With our problem set up, can we solve it using any of the supported solvers. We'll use ALTRO:

In [11]:
using ALTRO
opts = SolverOptions(
    penalty_scaling=100.,
    penalty_initial=0.1,
)

solver = ALTROSolver(prob, opts);
solve!(solver)
println("Cost: ", cost(solver))
println("Constraint violation: ", max_violation(solver))
println("Iterations: ", iterations(solver))

┌ Info: Precompiling ALTRO [5dcf52e5-e2fb-48e0-b826-96f46d2e3e73]
└ @ Base loading.jl:1260
┌ Warning: Package ALTRO does not have Statistics in its dependencies:
│ - If you have ALTRO checked out for development and have
│   added Statistics as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ALTRO
└ Loading Statistics into ALTRO from project dependency, future warnings for ALTRO are suppressed.


Cost: 0.2992834848479653
Constraint violation: 7.598400797448335e-10
Iterations: 90


## Visualizing the solution
We can use `TrajOptPlots` to visualize the solution:

In [12]:
using TrajOptPlots
using MeshCat
using Plots

vis = Visualizer()
render(vis)

┌ Info: Precompiling TrajOptPlots [7770976a-8dee-4930-bf39-a1782fd21ce6]
└ @ Base loading.jl:1260
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8701
└ @ MeshCat /home/bjack205/.julia/packages/MeshCat/ECbzr/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set(Any[]), ip"127.0.0.1", 8701))

For the visualization, we use `MeshIO v0.3` and `FileIO` to load in a mesh file. For the visualization, we need to tell `TrajOptPlots` what geometry to display, which we do by defining the `_set_mesh!` method for our model. Since our model is a `RigidBody`, `TrajOptPlots` already knows how to display it once the robot geometry is defined.

In [15]:
using FileIO, MeshIO
function TrajOptPlots._set_mesh!(vis, model::Quadrotor)
    obj = joinpath(@__DIR__, "quadrotor.obj")
    quad_scaling = 0.085
    robot_obj = FileIO.load(obj)
    robot_obj.vertices .*= quad_scaling
    mat = MeshPhongMaterial(color=colorant"black")
    setobject!(vis["geom"], robot_obj, mat)
end
TrajOptPlots.set_mesh!(vis, model)

MeshCat Visualizer with path /meshcat/robot/geom at http://localhost:8701

In [18]:
visualize!(vis, solver);